# ADSC W4: The Battle of Neighborhoods w.1

## Introduction

The report will outline how I have attempted to solve the business problem of comparing neighborhood similarity in different cities - in this case New York and Toronto. In addition to serving as a reference when comparing data, a “comparability-index” would also allow individuals interested in moving to new countries quantify how similar or dissimilar other neighborhoods or cities are to their current place of living. Some may be more interested in moving to something familiar (i.e. high “comparability-index”) while others are looking for a change (i.e. low “comparability-index”). My hope is that this report will show the usefulness of such a comparison.

## Data

I will use data from Foursquare for both New York and Tornoto in order to simplify data collection and comparability.

In [1]:
!pip install lxml
!pip install BeautifulSoup4
!pip install requests
!pip install geopy
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium
from geopy.geocoders import Nominatim
print('Finished')

     |████████████████████████████████| 5.5MB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 122kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 4.7MB/s eta 0:00:01
Finished


In [2]:
CLIENT_ID = 'JNIRIZZ52DHRGIUWSCMQYVVIXMJBGS0BK5EPCNLN0YWVFLFF' # your Foursquare ID
CLIENT_SECRET = 'SQRC1WDUPVHEADITO4A3ZH11TSLM50KTQSUHOKWHLB3FMKNA' # your Foursquare Secret
ACCESS_TOKEN = 'IJP1LPLBBVM325HTPLZDNQAYRC3MJJUZ5XAANXFOSXXFQPFQ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JNIRIZZ52DHRGIUWSCMQYVVIXMJBGS0BK5EPCNLN0YWVFLFF
CLIENT_SECRET:SQRC1WDUPVHEADITO4A3ZH11TSLM50KTQSUHOKWHLB3FMKNA


In [3]:
address_ny = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location_ny = geolocator.geocode(address_ny)
latitude_ny = location_ny.latitude
longitude_ny = location_ny.longitude
print(latitude_ny, longitude_ny)

40.7127281 -74.0060152


In [4]:
search_query = ''
radius = 1000
print(search_query + ' .... OK!')

 .... OK!


In [5]:
url_ny = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny ,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url_ny

'https://api.foursquare.com/v2/venues/search?client_id=JNIRIZZ52DHRGIUWSCMQYVVIXMJBGS0BK5EPCNLN0YWVFLFF&client_secret=SQRC1WDUPVHEADITO4A3ZH11TSLM50KTQSUHOKWHLB3FMKNA&ll=40.7127281,-74.0060152&oauth_token=IJP1LPLBBVM325HTPLZDNQAYRC3MJJUZ5XAANXFOSXXFQPFQ&v=20180604&query=&radius=1000&limit=30'

In [6]:
results_ny = requests.get(url_ny).json()
results_ny

{'meta': {'code': 200, 'requestId': '602bac884dcfcc005e25155b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4a676321f964a52051c91fe3',
    'name': 'New York City Hall',
    'location': {'address': '260 E Broadway',
     'crossStreet': 'at Murray St',
     'lat': 40.712658728604865,
     'lng': -74.00588035583496,
     'distance': 13,
     'postalCode': '10002',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['260 E Broadway (at Murray St)',
      'New York, NY 10002',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d129941735',
      'name': 'City Hall',
      'pluralName': 'City Halls',
      'shortName': 'City Hall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/cityhall_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '186051114'},
    'referralId': 'v-1613474952',
    '

In [7]:
# assign relevant part of JSON to venues
venues_ny = results_ny['response']['venues']

# tranform venues into a dataframe
dataframe_ny = json_normalize(venues_ny)
dataframe_ny.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.labeledLatLngs,location.neighborhood
0,4a676321f964a52051c91fe3,New York City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613474952,False,260 E Broadway,at Murray St,40.712659,-74.005880,13,10002,US,New York,NY,United States,"[260 E Broadway (at Murray St), New York, NY 1...",186051114,NaN,NaN
1,4b475390f964a520f12e26e3,Mary's Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1613474952,False,25-15 Queens Plz N,NaN,40.712786,-74.005944,8,11101,US,Long Island City,NY,United States,"[25-15 Queens Plz N, Long Island City, NY 1110...",NaN,"[{'label': 'display', 'lat': 40.71278567220636...",NaN
2,4b79a5e8f964a52037082fe3,NY Gift Shop,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",v-1613474952,False,234 Canal St,at Centre St.,40.717693,-73.999931,754,10013,US,New York,NY,United States,"[234 Canal St (at Centre St.), New York, NY 10...",NaN,"[{'label': 'display', 'lat': 40.71769332885742...",NaN
3,3fd66200f964a520d8f11ee3,City Hall Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1613474952,False,17 Park Row,btwn Broadway & Centre St,40.711893,-74.007792,176,10038,US,New York,NY,United States,"[17 Park Row (btwn Broadway & Centre St), New ...",NaN,NaN,NaN
4,5a5818e79e0d54787d1bc4d2,Estados Unidos,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1613474952,False,NaN,NaN,40.712776,-74.005974,6,10007,US,New York,NY,United States,"[New York, NY 10007, United States]",NaN,"[{'label': 'display', 'lat': 40.712776, 'lng':...",NaN


In [8]:
address_ca = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location_ca = geolocator.geocode(address_ca)
latitude_ca = location_ca.latitude
longitude_ca = location_ca.longitude
print(latitude_ca, longitude_ca)

34.0536909 -118.242766


In [9]:
url_ca = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ca, longitude_ca ,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url_ca

'https://api.foursquare.com/v2/venues/search?client_id=JNIRIZZ52DHRGIUWSCMQYVVIXMJBGS0BK5EPCNLN0YWVFLFF&client_secret=SQRC1WDUPVHEADITO4A3ZH11TSLM50KTQSUHOKWHLB3FMKNA&ll=34.0536909,-118.242766&oauth_token=IJP1LPLBBVM325HTPLZDNQAYRC3MJJUZ5XAANXFOSXXFQPFQ&v=20180604&query=&radius=1000&limit=30'

In [11]:
results_ca = requests.get(url_ca).json()
results_ca

{'meta': {'code': 200, 'requestId': '602bacead8aada4ab5ec4cca'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '513f84cee4b05b65d064fc57',
    'name': 'City Hall Observation Deck',
    'location': {'lat': 34.05362517781644,
     'lng': -118.24280276052687,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.05362517781644,
       'lng': -118.24280276052687}],
     'distance': 8,
     'cc': 'US',
     'city': 'Los Angeles',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['Los Angeles, CA', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d130941735',
      'name': 'Building',
      'pluralName': 'Buildings',
      'shortName': 'Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1613475050',
    'hasPerk': False},
   {'id': '4dd6d9cdd1647fcf3e3c1e67',
    'na

In [12]:
# assign relevant part of JSON to venues
venues = results_ca['response']['venues']

# tranform venues into a dataframe
dataframe_ca = json_normalize(venues)
dataframe_ca.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,...,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,513f84cee4b05b65d064fc57,City Hall Observation Deck,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",v-1613475050,False,34.053625,-118.242803,"[{'label': 'display', 'lat': 34.05362517781644...",8,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4dd6d9cdd1647fcf3e3c1e67,Homeboy Diner,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1613475050,False,34.053471,-118.242279,"[{'label': 'display', 'lat': 34.05347097170739...",51,US,...,2nd Floor City Hall,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4c5727dc15a39c7403323eb9,Mayor Tom Bradley Room,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.053785,-118.242761,"[{'label': 'display', 'lat': 34.05378544792505...",10,US,...,1st,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52ebf057498ee83c88d9c1f9,Los Angeles Mayor's Office of Economic Develop...,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1613475050,False,34.053787,-118.242369,"[{'label': 'display', 'lat': 34.05378746696942...",38,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b5113edf964a520314127e3,Los Angeles City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.053484,-118.242478,"[{'label': 'display', 'lat': 34.05348417688625...",35,US,...,at Temple Ave,90012,Civic Center,75727220,NaN,NaN,NaN,NaN,NaN,NaN
5,4df8f816b61ce821ebad4df2,L.A. City Hall Council Chambers,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.053713,-118.243324,"[{'label': 'display', 'lat': 34.05371331345282...",51,US,...,NaN,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4d9ccfabc593a1cd8dff5119,Office of Mayor Eric Garcetti,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.053931,-118.243169,"[{'label': 'display', 'lat': 34.05393149240509...",45,US,...,NaN,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4bccdd08511f952177ccb3c7,City Hall Planning Dept,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.053908,-118.242717,"[{'label': 'display', 'lat': 34.0539082131688,...",24,US,...,NaN,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4e2701211f6eb1ae139b0a96,Spring Street Forecourt - City Hall,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1613475050,False,34.053825,-118.242926,"[{'label': 'display', 'lat': 34.05382454395294...",20,US,...,Temple St.,90012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,527830a511d2b2f3e0481948,Budget and Finance Committee,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613475050,False,34.054067,-118.242780,"[{'label': 'display', 'lat': 34.05406725094131...",41,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_concat = [dataframe_ny, dataframe_ca]
df_concatecate = pd.concat(df_concat)
df_concatecate

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.distance,...,location.formattedAddress,venuePage.id,location.labeledLatLngs,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4a676321f964a52051c91fe3,New York City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613474952,False,260 E Broadway,at Murray St,40.712659,-74.005880,13,...,"[260 E Broadway (at Murray St), New York, NY 1...",186051114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b475390f964a520f12e26e3,Mary's Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1613474952,False,25-15 Queens Plz N,NaN,40.712786,-74.005944,8,...,"[25-15 Queens Plz N, Long Island City, NY 1110...",NaN,"[{'label': 'display', 'lat': 40.71278567220636...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b79a5e8f964a52037082fe3,NY Gift Shop,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",v-1613474952,False,234 Canal St,at Centre St.,40.717693,-73.999931,754,...,"[234 Canal St (at Centre St.), New York, NY 10...",NaN,"[{'label': 'display', 'lat': 40.71769332885742...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3fd66200f964a520d8f11ee3,City Hall Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1613474952,False,17 Park Row,btwn Broadway & Centre St,40.711893,-74.007792,176,...,"[17 Park Row (btwn Broadway & Centre St), New ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5a5818e79e0d54787d1bc4d2,Estados Unidos,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1613474952,False,NaN,NaN,40.712776,-74.005974,6,...,"[New York, NY 10007, United States]",NaN,"[{'label': 'display', 'lat': 40.712776, 'lng':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5d25d7fd3221cb002471e7c8,City Hall Plaza,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1613474952,False,Steve Flanders Square,Broadway & Centre Street,40.712641,-74.006131,13,...,[Steve Flanders Square (Broadway & Centre Stre...,NaN,"[{'label': 'display', 'lat': 40.712641, 'lng':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,51a4bc7c498e469047be66d6,City Hall Council Chambers,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1613474952,False,City Hall,NaN,40.712736,-74.005472,45,...,"[City Hall, New York, NY 10007, United States]",NaN,"[{'label': 'display', 'lat': 40.71273609783686...",Downtown Manhattan,NaN,NaN,NaN,NaN,NaN,NaN
7,4dbef8f60437955ec0721ff4,New York Film Academy,"[{'id': '4bf58dd8d48988d1a0941735', 'name': 'C...",v-1613474952,False,100 E 17th St,NaN,40.713040,-74.005381,63,...,"[100 E 17th St, New York, NY 10003, United Sta...",NaN,"[{'label': 'display', 'lat': 40.71303970231359...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4b76dba7f964a520b4642ee3,NY State Legislative Office Building,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1613474952,False,250 Broadway,NaN,40.712786,-74.005944,8,...,"[250 Broadway, New York, United States]",NaN,"[{'label': 'display', 'lat': 40.71278567220636...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4c584145744a0f4731c9cacb,Ocean Colony Beach,"[{'id': '4bf58dd8d48988d1e3941735', 'name': 'S...",v-1613474952,False,Montauk Hwy,NaN,40.712781,-74.005944,8,...,"[Montauk Hwy, Amaganset, NY, United States]",NaN,"[{'label': 'display', 'lat': 40.71278084415942...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
